# Tuto utilisation du cluster

Objectif de ce notebook est d'illustrer comment utiliser le clusteur de données pour accélérer des calculs d'analyse de données avec des données issus de base de type MongoDB et/ou Rasdaman

## I/ Architecture du cluster

### A/ Infrastructure physique

- 1 machine « Edge » ayant :
  - 128 G de RAM
  - 12 processeurs bi-cœurs
  - 4 * 2 To de disque HDD
- 2 machines « Master » ayant chacune :
  - 128 G de RAM
  - 8 processeurs bi-cœurs
  - 6 * 2 To de disque HDD
- 7 machines « Workers type 1 » ayant chacune :
  - 192 G de RAM
  - 12 processeurs bi-cœurs
  - 11 * 4 To de disque HDD
- 4 machines « Workers type 2 » ayant chacune :
  - 384 G de RAM
  - 14 processeurs bi-cœurs
  - 12 * 960 Go de disque SSD
- 1 machine « Worker type 3 » ayant :
  - 768 G de RAM
  - 14 processeurs bi-cœurs
  - 12 * 960 Go de disque SSD

### B/ Urbanisation du clusteur
Le cluster est composé des machines suivantes :
 
#### Machines Edge :
- edg01 :
  - Dropzone (disk 3)
  - NiFi (data disk 2)
  - Zeppelin  (http://islin-hdpmas1.ifp.fr:8999)
  - Superset
  - Grafana
  - Jupyter (http://islin-hdpledg01.ifp.fr)
  - Sqoop client
  - Oracle SQLplus
  - Knox Gateway
  - Ranger Admin
  - Rstudio
  - Ambari (https://islin-hdpmas01.ifp.fr:8080)

#### Machines Master :
- mas01 :
  - Namenode - active (disk 2, 3)
  - YARN NodeManager (active)
(disk 2, 3)
- mas02 :
  - Namenode - standby (disk 2, 3)
  - YARN NodeManager (standby)
  - Hive Server (logs sur le /disk6)
  - MongoDB : islin-hdplmas02.ifp.fr
      - islin-hdplnod01.ifp.fr
      - islin-hdplnod02.ifp.fr
      - islin-hdplnod03.ifp.fr
      - pymongo-client :  (edg01, nod01, nod02 et nod03)

#### Machines Worker :
- nod06, nod07, nod08, nod09
(type 1) :
  - Datanode (disk 2, 3, 4)
  - Kafka Broker (disk 5, 6, 7)
  - HBase RegionServer (disk 8, 9)
  - Zookeeper data dir (disk 10)
  - nod10, nod11, nod12 (type 1) :
  - Solr
  - Zookeeper for Solr
- nod01, nod02, nod03 (type 2) :
  - ArangoDB (disk 2, 3, 4)
  - MongoDB (disk 5, 6, 7)
- nod04 (type 2) :
  - Kubernetes :
  - InfluxDB
  - ChronoGraph https://islin-hdplnod04:8888
- nod05 (type 3) :
  - Datascience (nœud GPU)

#### Noms et adresse IP des noeuds

-  edg01 islin-hdpledg01.ifp.fr 10.126.51.131
-  mas01 islin-hdplmas01.ifp.fr 10.126.51.121
-  mas02 islin-hdplmas02.ifp.fr 10.126.51.122
-  nod01 islin-hdplnod01.ifp.fr 10.126.51.101
-  nod02 islin-hdplnod02.ifp.fr 10.126.51.102
-  nod03 islin-hdplnod03.ifp.fr 10.126.51.103
-  nod04 islin-hdplnod04.ifp.fr 10.126.51.104
-  nod05 islin-hdplnod05.ifp.fr 10.126.51.105
-  nod06 islin-hdplnod06.ifp.fr 10.126.51.106
-  nod07 islin-hdplnod07.ifp.fr 10.126.51.107
-  nod08 islin-hdplnod08.ifp.fr 10.126.51.108
-  nod09 islin-hdplnod09.ifp.fr 10.126.51.109
-  nod10 islin-hdplnod10.ifp.fr 10.126.51.110
-  nod11 islin-hdplnod11.ifp.fr 10.126.51.111
-  nod12 islin-hdplnod12.ifp.fr 10.126.51.112


## II/ HPC sur le cluster avec python et Dask

### A/ Configuration de l'environnement

#### 1) Prérequis

a) Avoir créer une répertoire /disk4/conda/<user> ou avoir accès à /home/irsrvshare2/R11/dgt_sandb/conda
    
b) Accès à conda :
-  /usr/local/Anaconda3-2019.03-Linux-x86_64
-  ou dans ~commonlib 
    - export MODULEPATH=~commonlib/ifpen/centos_7/modules/all:$MODULEPATH
    - module load anaconda3/2020.0 
    
```bash
export PATH=/usr/local/Anaconda3-2019.03-Linux-x86_64/bin:$PATH
export LD_LIBRARY_PATH=/usr/local/Anaconda3-2019.03-Linux-x86_64/lib:$LD_LIBRARY_PATH
export CONDA_ENVS_PATH=/disk4/conda/<user>/env
export CONDA_PKGS_DIRS=/disk4/conda/<user>/pkgs
export https_proxy=irproxy:8082
```


c) Initialiser conda :
    
```bash
conda init
conda config --set proxy_servers.http http://irproxy:8082
conda config --set proxy_servers.https https://irproxy:8082
```

#### 2) Création et packaging d'environnemnt conda utilisateur

Exemple environnement pour faire du dask, Rasdaman et traitement sur des images 3D


```bash
conda create -n img3D-env pthon=3.7
conda install -n img3D-env -c conda-forge [list des dependances packages ]
```

ou utilistation d'un fichier yaml du type img3D-environment.yml
```bash
name: img3D-env
channels:
  - conda-forge
dependencies:
  - python=3.7
  - jupyterlab>=2.0.0
  - numpy>=1.18.1
  - h5py
  - scipy>=1.3.0
  - toolz
  - dask=2.11.0
  - dask-labextension>=2.0.0
  - distributed=2.11.0
  - dask-yarn
  - matplotlib
  - pandas>=1.0.1
  - pandas-datareader
  - pytables
  - scikit-learn>=0.22.1
  - scikit-image>=0.15.0
  - ujson
  - pip
  - s3fs
```

Exemple de fichier pour rasdaman :
```bash
name: rasdaman-env
channels:
- conda-forge/label/cf202003
dependencies:
- python=3.7
- numpy
- pygrib
- jsonschema
- python-dateutil
- lxml
- grpcio
- protobuf
- matplotlib
```

Creation d'environnement à partir de fichier yaml :
```bash
conda env create -f img3D-environment.yml
```


Finalisation avec des packages spécifiques :
```bash
conda env list # pour avoir la liste des env disponibles
conda activate img3D-env

cd /path_to_rasdaman/rasdapy3
python setup.py install

cd /path_to_r11img/r11img
python setup.py install

cd /path_to_drp4m/drp4ml
python setup.py install
```

Packaging de l'environnement
```bash
conda pack -o img3d-env.tar.gz
conda deactivate
```

## C/ Simple soumission de job en mode Single Machine

La façon la plus simple d'utiliser Dask sur un neoud simple

Il suffit d'écrire puis lancer des scripts dask de la forme:

```bash
import dask as da
from dask.distributed import Client

client = Client(n_workers=4)

def square(x):
        return x ** 2

array = [ delayed(square)(i) for i in range(10))

results = da.compute(*array)

print('Results',results)
```

## D/ Soumission de job en mode Cluster

### 1) Gestion du cluster à a la main

Cela consiste à configurer manuellement un cluster dask avec =
- un scheduler
- plusieur worker sur différent noeuds du cluster

La difficulté est de bien s'assurer que sur chaque neuds, les environnements python sont bien accessibles et activés.

Il faut gérer aussi ces variables $PYTHONPATH pour avoir accès à tous ces modules


Exemple :
- un scheduler sur le noeuds islin-hpdledg01
- un worker avec 4 processus et 2 threads par process sur les noeuds islin-hpdlnod[06,07,08,09]
- un worker avec 4 processus et 1 GPU et 100GB de mémoire ilslin-hplnode05


```bash
> ssh islin-hpdledg01
> bash
> (base) conda activate img3D-env
> (img3D-env) dask-scheduler

Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the diagnostics dashboard on a random port instead.
  warnings.warn("\n" + msg)
distributed.scheduler - INFO -   Scheduler at:  tcp://10.126.51.131:8786
distributed.scheduler - INFO -   dashboard at:                    :40299
```

sur chaque noeuds islin-hpdlnod[06,07,08,09]

```bash
> ssh islin-hpdlnod06
> bash
> (base) conda activate img3D-env
> (img3D-env) dask-worker tcp://10.126.51.131:8786 --nprocs 4 --nthreads 2
```
sur le noeuds islin-hplnod05, lancement avec un GPU

```bash
> ssh islin-hpdlnod05
> bash
> (base) conda activate img3D-env
> (img3D-env) dask-worker tcp://10.126.51.131:8786 --nprocs 4 --resources "GPU=1" --resources "MEMORY=100e9"
```

on peut alors lancer des scripts dask de la forme :
```bash
from dask.distributed import Client
client = Client('10.126.51.131:8786')
def square(x):
        return x ** 2

def neg(x):
        return -x

A = client.map(square, range(10))
B = client.map(neg, A)
total = client.submit(sum, B)
print('RESULT:'total.result())
print('A=',client.gather(A)
```

Exemple de script nécessitant des ressources particulières de type GPU ou quantité mémoire.
```bash
from dask.distributed import Client
client = Client('10.126.51.131:8786')

data = [client.submit(load, fn) for fn in filenames]
processed = [client.submit(process, d, resources={'GPU': 1}) for d in data]
final = client.submit(aggregate, processed, resources={'MEMORY': 70e9})
```


### 2) Gestion du cluster en mode SSH

#### a) Prérequis

- configuration SSH sur tous les noeuds (clé, et authorized_key bien configuré)
- vérifier que l on peut se connecter d'un noeuds à l autre sans mot de passe
- module paramiko bien installé dans l environnement python, sinon
:
```bash
conda install -c conda-forge -n img3D-env paramiko asyncssh
```

#### b) Utilisation de l utilitaire dask-ssh

On peut utiliser l'utilitaire dask-ssh pour configurer le clusteur


On crée un fichier hostfile.txt avec un host pour le scheduler et une liste de host pour les workers

```bash
$ cat hostfile.txt
islin-hdpledg01.ifp.fr
islin-hdplnod06.ifp.fr 
islin-hdplnod07.ifp.fr 
islin-hdplnod08.ifp.fr 
islin-hdplnod09.ifp.fr 

$ dask-ssh --hostfile hostfile.txt --nprocs 4 --nthreads 2 --remote-python 'path_to_correct_python_from_user_env'
```

ensuite les scripts ont la forme suivante :

```bash
from dask.distributed import Client, SSHCluster

nodelist = ['edg01','nod06','nod07','nod08','nod09']
SSHCluster cluster(nodelist)
client = Client(cluster)

def square(x):
        return x ** 2

def neg(x):
        return -x

A = client.map(square, range(10))
B = client.map(neg, A)
total = client.submit(sum, B)
print('RESULT:'total.result())
print('A=',client.gather(A)
```

### 3) Gestion du cluster en mode YARN

#### a) Initialisation Kerberos

```bash
$ kinit user@HADOOP.IFP.FR -k -t /tmp/user.keytab
```

#### b) Forme des scripts dask

```bash
import time
import dask.array as da
from dask_yarn import YarnCluster
from dask.distributed import Client, progress

# Create a cluster where each worker has two cores and eight GiB of memory
cluster = YarnCluster(environment='img3D-env.tar.gz',
                      worker_vcores=2,
                      worker_memory="2GiB",
                      n_workers=6,
                      deploy_mode='local',
                      dashboard_address=':8789')
   
# Connect to the cluster
client = Client(cluster)

def square(x):
        return x ** 2

def neg(x):
        return -x

A = client.map(square, range(10))
B = client.map(neg, A)
total = client.submit(sum, B)
print('RESULT:'total.result())
print('A=',client.gather(A)

```

## III/ HPC sur le cluster avec python et Spark

### A/ Configuration de l'environnement

#### 1) Prérequis

a) Avoir créer une répertoire /disk4/conda/<user>
    
b) Accès à conda :
-  /usr/local/Anaconda3-2019.03-Linux-x86_64
-  ou dans ~commonlib : module load anaconda3/2020.0
    
```bash
export PATH=/usr/local/Anaconda3-2019.03-Linux-x86_64/bin:$PATH
export LD_LIBRARY_PATH=/usr/local/Anaconda3-2019.03-Linux-x86_64/lib:$LD_LIBRARY_PATH
export CONDA_ENVS_PATH=~dgt_sandb/conda/env
export CONDA_PKGS_DIRS=~dgt_sandb/pkgs
export https_proxy=irproxy:8082
```

c) Initialiser conda :
    
```bash
conda init
conda config --set proxy_servers.http http://irproxy:8082
conda config --set proxy_servers.https https://irproxy:8082
```

#### 2) Création et packaging d'environnemnt conda utilisateur

Exemple environnement pour faire pySpark, Rasdaman et traitement sur des images 3D


```bash
conda create -n img3D-spark-env pthon=3.7
conda install -n img3D-spark-env -c conda-forge [list des dependances packages ]
```

ou utilistation d'un fichier yaml du type img3D-environment.yml
```bash
name: img3D-spark-env
channels:
  - conda-forge
dependencies:
  - python=3.7
  - jupyterlab>=2.0.0
  - numpy>=1.18.1
  - h5py
  - scipy>=1.3.0
  - toolz
  - dask=2.11.0
  - dask-labextension>=2.0.0
  - distributed=2.11.0
  - dask-yarn
  - matplotlib
  - pandas>=1.0.1
  - pandas-datareader
  - pytables
  - scikit-learn>=0.22.1
  - scikit-image>=0.15.0
  - ujson
  - pip
  - s3fs
```

Exemple de fichier pour rasdaman :
```bash
name: rasdaman-env
channels:
- conda-forge/label/cf202003
dependencies:
- python=3.7
- numpy
- pygrib
- jsonschema
- python-dateutil
- lxml
- grpcio
- protobuf
- matplotlib
```

Creation d'environnement à partir de fichier yaml :
```bash
conda env create -f img3D-spark-environment.yml
```